In [ ]:
from pycellin.classes.feature import Feature
from pycellin.classes.feature_calculator import (
    FeatureCalculatorFactory,
    FeatureCalculator,
)

#### Global node feature: *absolute age*

In [ ]:
class AbsoluteAgeCalculator(FeatureCalculator):

    def compute(self, node, lineage):
        pass

In [ ]:
feat1 = Feature(
    "absolute_age",
    "Age of the cell since the beginning of the lineage",
    "CellLineage",
    "Pycellin",
    "int",  # "float" if in_time_unit else "int",
    "frame",  # self.metadata["time_unit"] if in_time_unit else "frame",
)

In [ ]:
factory = FeatureCalculatorFactory()
factory.register_calculator(feat1, AbsoluteAgeCalculator)

#### Local node feature: *node_ID parity*

In [ ]:
class ParityCalculator(FeatureCalculator):

    def compute(self, node):
        pass

In [ ]:
feat2 = Feature(
    "node_ID_parity",
    "Parity of the node ID",
    "CellLineage",
    "Pycellin",
    "int",
)

In [ ]:
factory = FeatureCalculatorFactory()
factory.register_calculator(feat2, ParityCalculator)